# Widget Visualization Test Notebook

This notebook tests the Phase 0 and Phase 1 widget implementation for visualizing structured prompts.

Expected behavior:
- First widget should include JavaScript bundle and KaTeX CSS (larger HTML output)
- Subsequent widgets should only include widget markup (smaller HTML output)
- Each widget should display three panes: Structure, Code View, and Preview

In [1]:
from t_prompts import dedent, prompt

## Test 1: Simple Prompt

This should display the first widget with full bundle injection.

In [2]:
task = "translate this to French"
p1 = prompt(t"Task: {task:t}")
p1

StructuredPrompt(keys=['t'], num_interpolations=1)

## Test 2: Second Simple Prompt

This should display without re-injecting the bundle (singleton pattern).

In [3]:
instruction = "Be concise and accurate"
p2 = prompt(t"Instructions: {instruction:i}")
p2

StructuredPrompt(keys=['i'], num_interpolations=1)

## Test 3: Nested Prompt

Test visualization of nested prompt structures.

In [4]:
system_msg = prompt(t"You are a helpful AI assistant.")
user_msg = "What is the capital of France?"
conversation = prompt(t"System: {system_msg:sys}\nUser: {user_msg:usr}")
conversation

StructuredPrompt(keys=['sys', 'usr'], num_interpolations=2)

## Test 4: Multi-line Prompt with Dedenting

In [5]:
task = "summarize"
topic = "climate change"
p4 = dedent(t"""
    You are an expert writer.

    Task: {task:t}
    Topic: {topic:top}

    Please provide a detailed response.
""")
p4

StructuredPrompt(keys=['t', 'top'], num_interpolations=2)

## Test 5: List Interpolation

Test visualization of list structures with separators.

In [6]:
examples = [
    prompt(t"Example 1: Simple addition"),
    prompt(t"Example 2: Complex multiplication"),
    prompt(t"Example 3: Division with remainder")
]
p5 = prompt(t"Here are some examples:\n{examples:ex:sep=\\n\\n}")
p5

StructuredPrompt(keys=['ex'], num_interpolations=1)

## Test 6: Render Hints - XML Wrapper

In [7]:
content = "This content will be wrapped in XML tags."
p6 = prompt(t"Content: {content:c:xml=document}")
p6

StructuredPrompt(keys=['c'], num_interpolations=1)

## Test 7: Render Hints - Markdown Header

In [8]:
section = "This is the introduction section."
p7 = prompt(t"{section:s:header=Introduction}")
p7

StructuredPrompt(keys=['s'], num_interpolations=1)

## Test 8: IntermediateRepresentation

Test visualization of rendered output with source mapping.

In [9]:
name = "Alice"
age = "30"
p8 = prompt(t"Name: {name:n}, Age: {age:a}")
ir = p8.render()
ir

IntermediateRepresentation(chunks=1, spans=4)

## Test 9: Complex Nested Structure

Test deeply nested prompts with multiple levels.

In [10]:
role = "teacher"
inner = prompt(t"Role: {role:r}")
middle = prompt(t"Context: {inner:ctx}")
outer = prompt(t"System: {middle:sys}")
outer

StructuredPrompt(keys=['sys'], num_interpolations=1)

## Test 10: Markdown with KaTeX

Test that KaTeX renders LaTeX math correctly in the preview pane.

In [11]:
formula = "The quadratic formula is: $x = \\frac{-b \\pm \\sqrt{b^2 - 4ac}}{2a}$"
p10 = prompt(t"{formula:f}")
p10

StructuredPrompt(keys=['f'], num_interpolations=1)

## Test 11: Combined Features

Test a realistic prompt with multiple features.

In [12]:
system = prompt(t"You are a helpful assistant specialized in mathematics.")
examples = [
    prompt(t"Q: What is 2+2?\nA: 4"),
    prompt(t"Q: What is 5*3?\nA: 15")
]
question = "What is the derivative of x^2?"

p11 = dedent(t"""
    {system:sys:xml=system}

    {examples:ex:header=Examples:sep=\\n\\n}

    Question: {question:q}
""")
p11

StructuredPrompt(keys=['sys', 'ex', 'q'], num_interpolations=3)

## Test 12: Check HTML Output Size

Verify that the singleton pattern is working by comparing HTML sizes.

In [ ]:
# Reset the bundle injection to test from fresh state
from t_prompts import widget_renderer

widget_renderer._bundle_injected = False

# First prompt - should include full bundle
p_first = prompt(t"First prompt")
html_first = p_first._repr_html_()
print(f"First widget HTML size: {len(html_first):,} bytes")
print(f"Contains bundle: {'tp-widget-bundle' in html_first}")

# Second prompt - should NOT include bundle
p_second = prompt(t"Second prompt")
html_second = p_second._repr_html_()
print(f"\nSecond widget HTML size: {len(html_second):,} bytes")
print(f"Contains bundle: {'tp-widget-bundle' in html_second}")

# Calculate size reduction
size_diff = len(html_first) - len(html_second)
size_pct = 100 * (1 - len(html_second) / len(html_first))
print(f"\nSize reduction: {size_diff:,} bytes ({size_pct:.1f}% smaller)")


## Test 13: Display Multiple Widgets at Once

Test that multiple widgets can coexist on the same page.

In [14]:
from IPython.display import display

p13a = prompt(t"Widget A")
p13b = prompt(t"Widget B")
p13c = prompt(t"Widget C")

display(p13a)
display(p13b)
display(p13c)

StructuredPrompt(keys=[], num_interpolations=0)

StructuredPrompt(keys=[], num_interpolations=0)

StructuredPrompt(keys=[], num_interpolations=0)

## Test 14: Source Location Display

Test that source location information is captured and displayed.

In [15]:
value = "test"
p14 = prompt(t"Value: {value:v}")

# Check that source location is captured
interp = p14['v']
if interp.source_location and interp.source_location.is_available:
    print(f"Source location captured: {interp.source_location.format_location()}")
else:
    print("Source location not available")

p14

Source location captured: 345218160.py:2


StructuredPrompt(keys=['v'], num_interpolations=1)

## Summary

If all tests above display correctly, the widget implementation is working as expected:

✅ **Phase 0 Features:**
- Singleton injection pattern (bundle only injected once)
- MutationObserver for dynamic cell rendering (Jupyter compatibility)
- CSS de-duplication

✅ **Phase 1 Features:**
- Three-pane static visualization
- Tree structure display
- **Code view shows actual rendered text** (not t-string syntax)
- **Images rendered as `<img>` tags** using base64 data
- Markdown preview with KaTeX rendering
- Support for nested prompts
- Support for list interpolations
- Render hints (XML, headers)
- IntermediateRepresentation visualization with chunks
- Source location tracking

## Expected Behavior

**For StructuredPrompt:**
- Structure pane: Tree showing all elements (statics, interpolations, nested prompts, lists)
- Code view: Shows the rendered text as if you called `str(prompt)`, with images as `<img>` tags
- Preview: Placeholder message (no preview until rendered)

**For IntermediateRepresentation:**
- Structure pane: Tree showing the source prompt structure
- Code view: Shows rendered output with text chunks and image chunks as inline `<img>` tags
- Preview: Markdown-rendered output with KaTeX math support

## Summary

If all tests above display correctly, the widget implementation is working as expected:

✅ **Phase 0 Features:**
- Singleton injection pattern (bundle only injected once)
- MutationObserver for dynamic cell rendering (Jupyter compatibility)
- CSS de-duplication

✅ **Phase 1 Features:**
- Three-pane static visualization
- Tree structure display
- **Code view shows actual rendered text** (not t-string syntax)
- **Images rendered as `<img>` tags** using base64 data
- **Markdown preview with KaTeX rendering** for LaTeX math
- Support for nested prompts
- Support for list interpolations
- Render hints (XML, headers)
- IntermediateRepresentation visualization with chunks
- Source location tracking

## Expected Behavior

**For StructuredPrompt:**
- **Structure pane**: Tree showing all elements (statics, interpolations, nested prompts, lists)
- **Code view**: Shows the rendered text as if you called `str(prompt)`, with images as inline `<img>` tags
- **Preview**: Markdown-rendered output with KaTeX math support (LaTeX rendered)

**For IntermediateRepresentation:**
- **Structure pane**: Tree showing the source prompt structure
- **Code view**: Shows rendered output with text chunks and image chunks as inline `<img>` tags
- **Preview**: Markdown-rendered output with KaTeX math support (LaTeX rendered)

## KaTeX Math Rendering

Both StructuredPrompt and IntermediateRepresentation preview panes support LaTeX math:
- Inline math: `$x = y$`
- Display math: `$$x = y$$`
- All KaTeX features supported (fractions, square roots, matrices, etc.)

In [ ]:
try:
    from PIL import Image

    # Create a simple test image
    img = Image.new('RGB', (100, 100), color='red')

    # Create prompt with image
    text = "Here is a red square:"
    p15 = prompt(t"{text:txt} {img:image}")

    print("Image prompt created. Rendering...")

    # Note: Images cannot be rendered to text, but they can be visualized in JSON
    # Let's just display the prompt structure
    display(p15)
except ImportError:
    print("PIL not installed - skipping image test")


## Test 15: Image Support (Optional)

If PIL is installed, test that images are rendered correctly in the code view.

**Note:** This test requires `pillow` to be installed. Skip if not available.